In [1]:
import pandas as pd
import gender_guesser.detector as gender
from pathlib import Path
DATA_DIR = Path('..')/'..'/'data'
pd.set_option('display.max_columns', 500)

In [2]:
def clean_first_name(table, colName):
    table['people'] = table['name'].str.split(',').str[1]
    table['first_name'] = table['people'].str.strip()
    table['first_name'] = table['first_name'].str.split(' ').str[0]
    table['first_name'].fillna('Entity', inplace = True)
    table = table.drop(columns = 'people', axis = 1)
    table['first_name'] = table['first_name'].str.title()
    return table


def combine_transactions(table):
    table['total_transaction'] = table['TRANSACTION_AMT'].groupby(table['NAME']).transform('sum')
    table['num_transactions'] = (table['CMTE_ID'].groupby(table['NAME']).transform('count'))
    table = table.drop(columns = 'TRANSACTION_AMT', axis = 1).drop_duplicates()
    table.columns = map(str.lower, table.columns)
    return table
        

def binarized_gender_table(rawTable, colName):
    genderDict = {'male':1, 'mostly_male':1, 'female':0, 'mostly_female':0, 'unknown':1}
    det = gender.Detector()
    table = clean_first_name(rawTable, colName)
    table['sex'] = table['first_name'].apply(det.get_gender)
    table['sex'] = table['sex'].map(genderDict)
    table = table.drop('first_name', axis = 1)
    return table

def clean_last_name(table, colName):
    table['people'] = table[colName].str.split(',').str[0]
    table['cand_last_name'] = table['people'].str.split(',').str[0]
    table['cand_last_name'] = table['cand_last_name'].str.strip()
    table['cand_last_name'].fillna('Entity', inplace = True)
    table['people'].fillna('Entity', inplace = True)
    table['cand_last_name'] = table['cand_last_name'].str.lower()
    table = table.drop('people', axis = 1)
    return table




In [28]:
# gov_final = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_governor_Final.csv')
# gov_final

In [3]:
indiv_names = ['CMTE_ID','AMNDT_IND','RPT_TP','TRANSACTION_PGI','IMAGE_NUM','TRANSACTION_TP','ENTITY_TP','NAME','CITY','STATE','ZIP_CODE','EMPLOYER','OCCUPATION','TRANSACTION_DT','TRANSACTION_AMT','OTHER_ID','TRAN_ID','FILE_NUM','MEMO_CD','MEMO_TEXT','SUB_ID']
indiv = pd.read_csv("s3://tenguins-tmp/fec/2018/itcont.txt", sep = "|", names = indiv_names )

/home/ec2-user/tenguins/.env/lib64/python3.7/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (10,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# cmte_colnames = pd.read_csv('../data/brandon/colnames/cm_header_file.csv')
# cmte_master = pd.read_csv('../data/brandon/cm.txt', sep='|', names=cmte_colnames.columns)

In [4]:
cand_colnames = pd.read_csv('../data/brandon/colnames/cn_header_file.csv')
cand_master = pd.read_csv('../data/brandon/cn.txt', sep='|', names=cand_colnames.columns)

In [5]:
linkage_colnames = pd.read_csv('../data/brandon/colnames/ccl_header_file.csv')
linkage = pd.read_csv('../data/brandon/ccl.txt', sep='|', names=linkage_colnames.columns)

In [6]:
# contrib_colnames = pd.read_csv('../data/brandon/colnames/pas2_header_file.csv')
# contrib = pd.read_csv('../data/brandon/itpas2.txt', sep='|', names=contrib_colnames.columns)

In [7]:
cand_names_to_merge = (cand_master.filter(['CAND_ID', 'CAND_NAME']))

In [8]:
indivs_to_linkage = pd.merge(indiv, linkage, on = 'CMTE_ID', how = 'inner')
indivs_with_cand_names = pd.merge(cand_names_to_merge, indivs_to_linkage, on = 'CAND_ID', how = 'inner')

In [9]:
# indivs_with_cand_names['CAND_ELECTION_YR'].unique()
indivs_with_cand_names = indivs_with_cand_names.drop(['CMTE_TP', 'SUB_ID', 'MEMO_TEXT', 'MEMO_CD', 'FILE_NUM',
                             'TRAN_ID', 'CMTE_DSGN', 'OTHER_ID', 'OCCUPATION', 'ZIP_CODE', 'CITY',
                            'ENTITY_TP', 'TRANSACTION_TP', 'IMAGE_NUM', 'RPT_TP', 'TRANSACTION_PGI'], axis = 1)

In [10]:
indivs_with_cand_names = indivs_with_cand_names.drop(['EMPLOYER'], axis = 1)

In [11]:
indivs_comb_transactions = combine_transactions(indivs_with_cand_names)

In [50]:
indivs_comb_transactions

,cand_id,cand_name,cmte_id,amndt_ind,name,state,transaction_dt,cand_election_yr,fec_election_yr,linkage_id,total_transaction,num_transactions
0,H0AL02087,"ROBY, MARTHA",C00462143,N,"HINRICHS-WILSON, KELLY R. MRS",AL,3312017.0,2018,2018,212429,8100.0,3.0
1,H0AL02087,"ROBY, MARTHA",C00462143,N,"KEEN, JASON M.",AL,3152017.0,2018,2018,212429,3700.0,2.0
2,H0AL02087,"ROBY, MARTHA",C00462143,N,"LEGGIERI, JOHN A.",VA,3282017.0,2018,2018,212429,8500.0,9.0
3,H0AL02087,"ROBY, MARTHA",C00462143,N,"COTTER, BILLY G.",AL,3162017.0,2018,2018,212429,2000.0,2.0
4,H0AL02087,"ROBY, MARTHA",C00462143,N,"COTTER, ROSS R. JR.",AL,3162017.0,2018,2018,212429,1000.0,1.0
5,H0AL02087,"ROBY, MARTHA",C00462143,N,"DAVIDSON, DOROTHY S. MRS",AL,3152017.0,2018,2018,212429,7000.0,4.0
6,H0AL02087,"ROBY, MARTHA",C00462143,N,"DUDLEY, JOHN ROBERT III",AL,1032017.0,2018,2018,212429,4000.0,3.0
7,H0AL02087,"ROBY, MARTHA",C00462143,N,"GASTRIGHT, JOHN A. JR",MD,3312017.0,2018,2018,212429,250.0,1.0
8,H0AL02087,"ROBY, MARTHA",C00462143,N,"GESSLER, CARL JR.",AL,3172017.0,2018,2018,212429,1500.0,2.0
9,H0AL02087,"ROBY, MARTHA",C00462143,N,"GREEN, JEFFERY A.",MD,2132017.0,2018,2018,212429,2500.0,5.0


In [12]:
indivs_with_gender = binarized_gender_table(indivs_comb_transactions, 'name')

In [13]:
indivs_with_gender = clean_last_name(indivs_with_gender, 'cand_name')

In [14]:
indivs_with_gender.to_pickle('fat_indivs_plus_polls.pkl')

In [59]:
# indivs_to_linkage.isnull().sum()

In [58]:
cand_master.head()

,CAND_ID,CAND_NAME,CAND_PTY_AFFILIATION,CAND_ELECTION_YR,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP
0,H0AL02087,"ROBY, MARTHA",REP,2018,AL,H,2.0,I,C,C00462143,3260 BANKHEAD AVE,NaN,MONTGOMERY,AL,361062448.0
1,H0AL03192,"THOMPSON, HANNAH",DEM,2020,AL,H,3.0,NaN,N,C00681452,2181 N BROADWAY,NaN,ALEXANDER CITY,AL,35010.0
2,H0AL05049,"CRAMER, ROBERT E ""BUD"" JR",DEM,2008,AL,H,5.0,NaN,P,C00239038,PO BOX 2621,NaN,HUNTSVILLE,AL,35804.0
3,H0AL05163,"BROOKS, MO",REP,2018,AL,H,5.0,I,C,C00464149,7610 FOXFIRE DR.,NaN,HUNTSVILLE,AL,35802.0
4,H0AL06088,"COOKE, STANLEY KYLE",REP,2010,AL,H,6.0,C,N,C00464222,723 CHERRY BROOK ROAD,NaN,KIMBERLY,AL,35091.0


In [10]:
linkage.head()

,CAND_ID,CAND_ELECTION_YR,FEC_ELECTION_YR,CMTE_ID,CMTE_TP,CMTE_DSGN,LINKAGE_ID
0,C00008227,2016,2018,C00623207,N,J,212420
1,C00018929,2017,2018,C00565861,N,J,218749
2,C00027466,2016,2018,C00623207,N,J,212421
3,C00027466,2018,2018,C00682112,N,J,221609
4,C00042366,2016,2018,C00624478,N,J,212422


In [11]:
contrib.head()

,CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,CAND_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,C00177436,N,M2,P,201702039042410900,24K,CCM,"YODER FOR CONGRESS, INC",OVERLAND PARK,KS,66225,NaN,NaN,1172017.0,1500,C00472365,H0KS03137,40458934,1147350,NaN,NaN,4020820171369963567
1,C00242271,N,M2,P,201702069044298709,24K,CCM,KEVIN MCCARTHY FOR CONGRESS,BAKERSFIELD,CA,933892667,NaN,NaN,1182017.0,5000,C00420935,H6CA22125,74716003,1147780,NaN,CONTRIBUTION,4020820171369964484
2,C00242271,N,M2,P,201702069044298709,24K,CCM,TIBERI FOR CONGRESS,COLUMBUS,OH,432312098,NaN,NaN,1182017.0,5000,C00347492,H0OH12062,74716004,1147780,NaN,CONTRIBUTION,4020820171369964486
3,C00242271,N,M2,P,201702069044298710,24K,CCM,JEFF FLAKE FOR US SENATE INC,TEMPE,AZ,852840042,NaN,NaN,1182017.0,2500,C00347260,S2AZ00141,74716067,1147780,NaN,CONTRIBUTION,4020820171369964489
4,C00051979,N,M2,P,201702069044298770,24K,CCM,KAINE FOR VIRGINIA,ALEXANDRIA,VA,22314,NaN,NaN,1232017.0,1000,C00495358,S2VA00142,8481131,1147789,NaN,CONTRIBUTION,4020820171369964549


In [62]:
# masterB =(
#     linkage.merge(cmte_master, on=['CMTE_ID', 'CAND_ID'], suffixes=('_linkage', '_cmte'), how='right')
#     .merge(cand_master, on=['CAND_ID', 'CAND_ELECTION_YR'], how='right')
#     .merge(contrib, left_on=['CMTE_ID', 'CAND_ID', 'CMTE_NM'], right_on=['CMTE_ID', 'CAND_ID', 'NAME'], how='right')
# )

In [63]:
# masterB.info()

In [46]:
master = (
    linkage.merge(cmte_master, on=['CMTE_ID', 'CAND_ID', 'CMTE_TP', 'CMTE_DSGN'], suffixes=('_linkage', '_cmte'), how='right')
    .merge(cand_master, on=['CAND_ID', 'CAND_ELECTION_YR'], how='right')
    .merge(contrib, left_on=['CMTE_ID', 'CAND_ID', 'CMTE_NM'], right_on=['CMTE_ID', 'CAND_ID', 'NAME'], how='right')
)

In [47]:
master

,CAND_ID,CAND_ELECTION_YR,FEC_ELECTION_YR,CMTE_ID,CMTE_TP,CMTE_DSGN,LINKAGE_ID,CMTE_NM,TRES_NM,CMTE_ST1,CMTE_ST2,CMTE_CITY,CMTE_ST,CMTE_ZIP,CMTE_PTY_AFFILIATION,CMTE_FILING_FREQ,ORG_TP,CONNECTED_ORG_NM,CAND_NAME,CAND_PTY_AFFILIATION,CAND_OFFICE_ST,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ICI,CAND_STATUS,CAND_PCC,CAND_ST1,CAND_ST2,CAND_CITY,CAND_ST,CAND_ZIP,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
0,H2OH15046,2008.0,2018.0,C00265850,H,P,213288.0,PRYCE FOR CONGRESS,"PRYCE, DEBORAH",250 WEST STREET,NaN,COLUMBUS,OH,43215,REP,Q,NaN,NONE,"PRYCE, DEBORAH D.",REP,OH,H,15.0,NaN,P,C00265850,145 EAST RICH STREET,NaN,COLUMBUS,OH,43215.0,N,Q2,P,201807129115383064,24K,CCM,PRYCE FOR CONGRESS,COLUMBUS,OH,43215,NaN,NaN,6082018.0,2000,C00265850,SB21.4113,1244759,NaN,NaN,4071320181575869312
1,H8AK00132,2018.0,2018.0,C00646521,H,P,217273.0,DIMITRI FOR CONGRESS,SERGEY A PETROV,PO BOX 90787,NaN,ANCHORAGE,AK,99509,DEM,Q,NaN,NaN,"SHEIN, DIMITRI",DEM,AK,H,0.0,C,C,C00646521,PO BOX 90787,NaN,ANCHORAGE,AK,99509.0,N,Q2,P,201807149115470250,24C,CCM,DIMITRI FOR CONGRESS,ANCHORAGE,AK,99509,NaN,NaN,6302018.0,316,H8AK00132,SB19A.5219,1246838,NaN,NaN,4071520181575965675
2,H8CA26130,2018.0,2018.0,C00655282,H,P,217993.0,JOHN NELSON FOR CONGRESS,"NELSON, VICTORIA",2081 N. OXNARD BLVD. #111,NaN,OXNARD,CA,93036,DEM,Q,NaN,NONE,"NELSON, JOHN",DEM,CA,H,26.0,C,C,C00655282,2081 N OXNARD BLVD,APT 111,OXNARD,CA,93036.0,N,YE,P,201801319091298808,24K,CCM,JOHN NELSON FOR CONGRESS,OXNARD,CA,93036,NaN,NaN,12182017.0,200,C00655282,SB21.4170,1205529,NaN,NaN,4020520181503828581
3,H8GA02079,2018.0,2018.0,C00664177,H,P,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,"CROUCH, PAUL",2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907,DEM,Q,NaN,NONE,"SPRAYBERRY, CARL",DEM,GA,H,2.0,C,N,C00664177,2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907.0,N,Q1,P,201804159108221971,24E,PAC,MILLENNIAL REFORMATION OF THE HOUSE,COLUMBUS,GA,31907,NaN,NaN,3312018.0,0,H8GA02079,SE.4123,1223526,NaN,NaN,4041820181526083951
4,H8GA02079,2018.0,2018.0,C00664177,H,P,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,"CROUCH, PAUL",2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907,DEM,Q,NaN,NONE,"SPRAYBERRY, CARL",DEM,GA,H,2.0,C,N,C00664177,2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907.0,N,Q1,P,201804159108221971,24E,PAC,MILLENNIAL REFORMATION OF THE HOUSE,COLUMBUS,GA,31907,NaN,NaN,3312018.0,0,H8GA02079,SE.4124,1223526,NaN,NaN,4041820181526083953
5,H8GA02079,2018.0,2018.0,C00664177,H,P,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,"CROUCH, PAUL",2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907,DEM,Q,NaN,NONE,"SPRAYBERRY, CARL",DEM,GA,H,2.0,C,N,C00664177,2920 NORRIS ROAD,NaN,COLUMBUS,GA,31907.0,N,Q1,P,201804159108221972,24E,PAC,MILLENNIAL REFORMATION OF THE HOUSE,COLUMBUS,GA,31907,NaN,NaN,3312018.0,0,H8GA02079,SE.4139,1223526,NaN,NaN,4041820181526083955
6,H8PA05279,2018.0,2018.0,C00671867,H,P,219825.0,KIRKLAND FOR CONGRESS,"SMITH, CLIFTON J. II",403 AVENUYE OF THE STATES,SUITE 2-B,CHESTER,PA,19013,DEM,Q,NaN,NONE,"KIRKLAND, THADDEUS",DEM,PA,H,5.0,C,C,C00671867,1027-1/2 W. 8TH STREET,NaN,CHESTER,PA,19013.0,N,Q2,P,201807159115650695,24K,CCM,KIRKLAND FOR CONGRESS,CHESTER,PA,19013,NaN,NaN,5142018.0,5000,C00671867,SB21.4323,1248464,NaN,NaN,4071720181576136681
7,H8SC04227,2018.0,2018.0,C00666230,H,P,219281.0,LEE TURNER FOR CONGRESS,"THOMPSON, ANN",PO BOX 8248,NaN,GREENVILLE,SC,29604,DEM,T,NaN,NaN,"TURNER, D LEE",DEM,SC,H,4.0,O,C,C00666230,21 TWIN SPRINGS DR,NaN,GREENVILLE,SC,29605.0,T,TER,R,201810159125409401,24K,CCM,LEE TURNER FOR CONGRESS,NaN,NaN,NaN,NaN,NaN,9272018.0,2122,C00666230,SB21.4098,1272855,NaN,NaN,4110920181616409188
8,H0KS03137,NaN,NaN,C00177436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,M2,P,201702039042410900,24K,CCM,"YODER FOR CONGRESS, INC",OVERLAND PARK,KS,66225,NaN,NaN,1172017.0,1500,C00472365,40458934,1147350,NaN,NaN,4020820171369963567
9,H6CA

In [48]:
master.columns

Index(['CAND_ID', 'CAND_ELECTION_YR', 'FEC_ELECTION_YR', 'CMTE_ID', 'CMTE_TP',
       'CMTE_DSGN', 'LINKAGE_ID', 'CMTE_NM', 'TRES_NM', 'CMTE_ST1', 'CMTE_ST2',
       'CMTE_CITY', 'CMTE_ST', 'CMTE_ZIP', 'CMTE_PTY_AFFILIATION',
       'CMTE_FILING_FREQ', 'ORG_TP', 'CONNECTED_ORG_NM', 'CAND_NAME',
       'CAND_PTY_AFFILIATION', 'CAND_OFFICE_ST', 'CAND_OFFICE',
       'CAND_OFFICE_DISTRICT', 'CAND_ICI', 'CAND_STATUS', 'CAND_PCC',
       'CAND_ST1', 'CAND_ST2', 'CAND_CITY', 'CAND_ST', 'CAND_ZIP', 'AMNDT_IND',
       'RPT_TP', 'TRANSACTION_PGI', 'IMAGE_NUM', 'TRANSACTION_TP', 'ENTITY_TP',
       'NAME', 'CITY', 'STATE', 'ZIP_CODE', 'EMPLOYER', 'OCCUPATION',
       'TRANSACTION_DT', 'TRANSACTION_AMT', 'OTHER_ID', 'TRAN_ID', 'FILE_NUM',
       'MEMO_CD', 'MEMO_TEXT', 'SUB_ID'],
      dtype='object')

In [49]:
master.isnull().sum()

CAND_ID                   1326
CAND_ELECTION_YR        301731
FEC_ELECTION_YR         301731
CMTE_ID                      0
CMTE_TP                 301731
CMTE_DSGN               301731
LINKAGE_ID              301731
CMTE_NM                 301731
TRES_NM                 301731
CMTE_ST1                301731
CMTE_ST2                301738
CMTE_CITY               301731
CMTE_ST                 301731
CMTE_ZIP                301731
CMTE_PTY_AFFILIATION    301731
CMTE_FILING_FREQ        301731
ORG_TP                  301739
CONNECTED_ORG_NM        301733
CAND_NAME               301731
CAND_PTY_AFFILIATION    301731
CAND_OFFICE_ST          301731
CAND_OFFICE             301731
CAND_OFFICE_DISTRICT    301731
CAND_ICI                301732
CAND_STATUS             301731
CAND_PCC                301731
CAND_ST1                301731
CAND_ST2                301738
CAND_CITY               301731
CAND_ST                 301731
CAND_ZIP                301731
AMNDT_IND                    0
RPT_TP  

In [50]:
master = master.drop(columns = ['OTHER_ID', 'TRAN_ID', 'FILE_NUM', 'MEMO_CD', 'MEMO_TEXT', 'SUB_ID', 'ZIP_CODE',
 'EMPLOYER', 'OCCUPATION','RPT_TP', 'TRANSACTION_PGI', 'IMAGE_NUM', 'CAND_ICI', 'CMTE_FILING_FREQ','TRANSACTION_DT',
                                'CAND_OFFICE_ST', 'CONNECTED_ORG_NM','CMTE_ST1', 'CMTE_ST2', 'TRES_NM', 'CAND_STATUS',
                                'CAND_PCC', 'CAND_ST1', 'CAND_ST2', 'ORG_TP', 'CMTE_CITY', 'CMTE_ZIP', 'TRANSACTION_TP',
                               'CITY', 'CAND_CITY', 'CMTE_DSGN'])

In [52]:
master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301739 entries, 0 to 301738
Data columns (total 20 columns):
CAND_ID                 300413 non-null object
CAND_ELECTION_YR        8 non-null float64
FEC_ELECTION_YR         8 non-null float64
CMTE_ID                 301739 non-null object
CMTE_TP                 8 non-null object
LINKAGE_ID              8 non-null float64
CMTE_NM                 8 non-null object
CMTE_ST                 8 non-null object
CMTE_PTY_AFFILIATION    8 non-null object
CAND_NAME               8 non-null object
CAND_PTY_AFFILIATION    8 non-null object
CAND_OFFICE             8 non-null object
CAND_OFFICE_DISTRICT    8 non-null float64
CAND_ST                 8 non-null object
CAND_ZIP                8 non-null float64
AMNDT_IND               301739 non-null object
ENTITY_TP               298207 non-null object
NAME                    301494 non-null object
STATE                   301204 non-null object
TRANSACTION_AMT         301739 non-null int64
dtypes: fl

In [55]:
master

,CAND_ID,CAND_ELECTION_YR,FEC_ELECTION_YR,CMTE_ID,CMTE_TP,LINKAGE_ID,CMTE_NM,CMTE_ST,CMTE_PTY_AFFILIATION,CAND_NAME,CAND_PTY_AFFILIATION,CAND_OFFICE,CAND_OFFICE_DISTRICT,CAND_ST,CAND_ZIP,AMNDT_IND,ENTITY_TP,NAME,STATE,TRANSACTION_AMT
0,H2OH15046,2008.0,2018.0,C00265850,H,213288.0,PRYCE FOR CONGRESS,OH,REP,"PRYCE, DEBORAH D.",REP,H,15.0,OH,43215.0,N,CCM,PRYCE FOR CONGRESS,OH,2000
1,H8AK00132,2018.0,2018.0,C00646521,H,217273.0,DIMITRI FOR CONGRESS,AK,DEM,"SHEIN, DIMITRI",DEM,H,0.0,AK,99509.0,N,CCM,DIMITRI FOR CONGRESS,AK,316
2,H8CA26130,2018.0,2018.0,C00655282,H,217993.0,JOHN NELSON FOR CONGRESS,CA,DEM,"NELSON, JOHN",DEM,H,26.0,CA,93036.0,N,CCM,JOHN NELSON FOR CONGRESS,CA,200
3,H8GA02079,2018.0,2018.0,C00664177,H,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,GA,DEM,"SPRAYBERRY, CARL",DEM,H,2.0,GA,31907.0,N,PAC,MILLENNIAL REFORMATION OF THE HOUSE,GA,0
4,H8GA02079,2018.0,2018.0,C00664177,H,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,GA,DEM,"SPRAYBERRY, CARL",DEM,H,2.0,GA,31907.0,N,PAC,MILLENNIAL REFORMATION OF THE HOUSE,GA,0
5,H8GA02079,2018.0,2018.0,C00664177,H,218995.0,MILLENNIAL REFORMATION OF THE HOUSE,GA,DEM,"SPRAYBERRY, CARL",DEM,H,2.0,GA,31907.0,N,PAC,MILLENNIAL REFORMATION OF THE HOUSE,GA,0
6,H8PA05279,2018.0,2018.0,C00671867,H,219825.0,KIRKLAND FOR CONGRESS,PA,DEM,"KIRKLAND, THADDEUS",DEM,H,5.0,PA,19013.0,N,CCM,KIRKLAND FOR CONGRESS,PA,5000
7,H8SC04227,2018.0,2018.0,C00666230,H,219281.0,LEE TURNER FOR CONGRESS,SC,DEM,"TURNER, D LEE",DEM,H,4.0,SC,29605.0,T,CCM,LEE TURNER FOR CONGRESS,NaN,2122
8,H0KS03137,NaN,NaN,C00177436,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,CCM,"YODER FOR CONGRESS, INC",KS,1500
9,H6CA22125,NaN,NaN,C00242271,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,CCM,KEVIN MCCARTHY FOR CONGRESS,CA,5000


In [54]:
master.columns

Index(['CAND_ID', 'CAND_ELECTION_YR', 'FEC_ELECTION_YR', 'CMTE_ID', 'CMTE_TP',
       'LINKAGE_ID', 'CMTE_NM', 'CMTE_ST', 'CMTE_PTY_AFFILIATION', 'CAND_NAME',
       'CAND_PTY_AFFILIATION', 'CAND_OFFICE', 'CAND_OFFICE_DISTRICT',
       'CAND_ST', 'CAND_ZIP', 'AMNDT_IND', 'ENTITY_TP', 'NAME', 'STATE',
       'TRANSACTION_AMT'],
      dtype='object')

In [1]:
indiv.info()

NameError: name 'indiv' is not defined

In [70]:
indivs_with_gender.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3459829 entries, 0 to 3870004
Data columns (total 14 columns):
cand_id              object
cand_name            object
cmte_id              object
amndt_ind            object
name                 object
state                object
transaction_dt       float64
cand_election_yr     int64
fec_election_yr      int64
linkage_id           int64
total_transaction    float64
num_transactions     float64
sex                  float64
cand_last_name       object
dtypes: float64(4), int64(3), object(7)
memory usage: 395.9+ MB
